In [17]:
#Para mostrar sugerencias con TAB
!pip install ipython
%config Completer.use_jedi = False

In [ ]:
#Para autocompletar
!pip install jupyter_contrib_nbextensions

In [ ]:
!pip install jupyter_nbextensions_configurator

In [ ]:
!jupyter contrib nbextensions install --user

# Clase 0824

In [1]:
! pip install findspark 
#Para que python encuentre spark

In [2]:
! pip install pyspark 
#Instala la api que me permite acceder al pyspark que ya instalé

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Pyspark")
sc = SparkContext(conf = conf)

In [5]:
file = sc.textFile("artofwar.txt")
file.collect()

['The Art of War', 'A Book of Strategy', 'By Sun, Tzu']

In [ ]:
file.map(lambda x: x.lower() )\
    .collect()

In [ ]:
movilens = sc.textFile("./ml-1m/ratings.dat")
movilens.first()

# Promedio

In [ ]:
#Se inicia el findspark
import findspark
findspark.init()

#Se configura el spark
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Pyspark")
sc = SparkContext(conf = conf)

In [ ]:
def parseLines(x): #UserID::MovieID::Rating::Timestamp
  x = x.split('::')[1:3] #[1:3] me muestra los dos primeros datos del arreglo
  x[1] = float(x[1])
  return(tuple(x)) # Para par clave por lo general se representa en tuplas

In [ ]:
movilens = sc.textFile("./ml-1m/ratings.dat")
movilens.first()

In [ ]:
ratings = movilens.map(parseLines)\
                  .mapValues(lambda x: (x, 1))\
                  .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                  .mapValues(lambda x: x[0] / x[1])\
                  .sortBy(lambda x: -x[1])

# .map(parseLines)\
# .mapValues(lambda x: (x, 1))\  #Convierte cada rating en una tupla (rating, 1) -->
# .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\ #Suma los ratings y cuenta cuántos hay.
# .mapValues(lambda x: x[0] / x[1])\ #Calcula el prom
# .sortBy(lambda x: -x[1])

In [ ]:
rta = ratings.collect()
for movie, rating_prom in rta:
    print(f"MovieID: {movie}, Rating promedio: {rating_prom}")

# Clase 0907

In [7]:
#Se inicia el findspark
import findspark
findspark.init()

#Se configura el spark
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Pyspark")
sc = SparkContext(conf = conf)

In [8]:
def parseLines(x):
  x = x.split('::')[1:3] #[1:3] me muestra los dos primeros datos del arreglo
  x[1] = float(x[1])
  return(tuple(x)) # Para par clave por lo general se representa en tuplas

  #(reduceByKey)hace la operación en las claves (películas) que son iguales
movilens = sc.textFile("./ml-1m/ratings.dat")

ratings = movilens.map(parseLines)\
                  .mapValues(lambda x: (x,1))\
                  .reduceByKey(lambda x , y: (x[0] + y[0], x[1] + y[1]))\
                  .mapValues(lambda x: x[0]/x[1])\
                  .sortBy(lambda x: -x[1])
ratings.collect()

[('787', 5.0),
 ('1830', 5.0),
 ('3607', 5.0),
 ('3280', 5.0),
 ('989', 5.0),
 ('3881', 5.0),
 ('3233', 5.0),
 ('3172', 5.0),
 ('3656', 5.0),
 ('3382', 5.0),
 ('3245', 4.8),
 ('53', 4.75),
 ('2503', 4.666666666666667),
 ('2905', 4.608695652173913),
 ('2019', 4.560509554140127),
 ('318', 4.554557700942973),
 ('858', 4.524966261808367),
 ('745', 4.52054794520548),
 ('50', 4.517106001121705),
 ('527', 4.510416666666667),
 ('1148', 4.507936507936508),
 ('1795', 4.5),
 ('3517', 4.5),
 ('557', 4.5),
 ('2480', 4.5),
 ('2309', 4.5),
 ('578', 4.5),
 ('3888', 4.5),
 ('439', 4.5),
 ('922', 4.491489361702127),
 ('1198', 4.477724741447892),
 ('904', 4.476190476190476),
 ('1178', 4.473913043478261),
 ('260', 4.453694416583082),
 ('1212', 4.452083333333333),
 ('750', 4.4498902706656915),
 ('3338', 4.444444444444445),
 ('720', 4.426940639269406),
 ('1207', 4.425646551724138),
 ('3435', 4.415607985480944),
 ('912', 4.412822049131217),
 ('670', 4.410714285714286),
 ('2762', 4.406262708418057),
 ('3030',

In [9]:
def get_movies_info():
    #MovieID::Title::Genres
    movieNames = {}
    
    with open("./ml-1m/movies.dat" , "r", encoding = "latin1") as f:
        for line in f:
            x = line.split("::")
            movieNames[x[0]] = {"name" : x[1],
                                "genres": x[2].replace("\n", "")}
    return movieNames

In [10]:
movieDictio = sc.broadcast(get_movies_info())
movieDictio.value["1"]

{'name': 'Toy Story (1995)', 'genres': "Animation|Children's|Comedy"}

In [11]:
def get_users_info():
    #UserID::Gender::Age::Occupation::Zip-code
    usersInfo = {}
    
    with open("./ml-1m/users.dat" , "r", encoding = "latin1") as f:
        for line in f:
            x = line.split("::")
            usersInfo[x[0]] = {"gender" : x[1],
                               "Age" : x[2],
                                "Occupation": x[3]}
    return usersInfo

In [12]:
userDictio = sc.broadcast(get_users_info())
userDictio.value

{'1': {'gender': 'F', 'Age': '1', 'Occupation': '10'},
 '2': {'gender': 'M', 'Age': '56', 'Occupation': '16'},
 '3': {'gender': 'M', 'Age': '25', 'Occupation': '15'},
 '4': {'gender': 'M', 'Age': '45', 'Occupation': '7'},
 '5': {'gender': 'M', 'Age': '25', 'Occupation': '20'},
 '6': {'gender': 'F', 'Age': '50', 'Occupation': '9'},
 '7': {'gender': 'M', 'Age': '35', 'Occupation': '1'},
 '8': {'gender': 'M', 'Age': '25', 'Occupation': '12'},
 '9': {'gender': 'M', 'Age': '25', 'Occupation': '17'},
 '10': {'gender': 'F', 'Age': '35', 'Occupation': '1'},
 '11': {'gender': 'F', 'Age': '25', 'Occupation': '1'},
 '12': {'gender': 'M', 'Age': '25', 'Occupation': '12'},
 '13': {'gender': 'M', 'Age': '45', 'Occupation': '1'},
 '14': {'gender': 'M', 'Age': '35', 'Occupation': '0'},
 '15': {'gender': 'M', 'Age': '25', 'Occupation': '7'},
 '16': {'gender': 'F', 'Age': '35', 'Occupation': '0'},
 '17': {'gender': 'M', 'Age': '50', 'Occupation': '1'},
 '18': {'gender': 'F', 'Age': '18', 'Occupation': '

In [13]:
ratings.filter(lambda x: x[1] > 4.5)\
               .map(lambda x: (movieDictio.value[x[0]]["name"], x[1]))\
               .collect()

[('Gate of Heavenly Peace, The (1995)', 5.0),
 ('Follow the Bitch (1998)', 5.0),
 ('One Little Indian (1973)', 5.0),
 ('Baby, The (1973)', 5.0),
 ('Schlafes Bruder (Brother of Sleep) (1995)', 5.0),
 ('Bittersweet Motel (2000)', 5.0),
 ('Smashing Time (1967)', 5.0),
 ('Ulysses (Ulisse) (1954)', 5.0),
 ('Lured (1947)', 5.0),
 ('Song of Freedom (1936)', 5.0),
 ('I Am Cuba (Soy Cuba/Ya Kuba) (1964)', 4.8),
 ('Lamerica (1994)', 4.75),
 ('Apple, The (Sib) (1998)', 4.666666666666667),
 ('Sanjuro (1962)', 4.608695652173913),
 ('Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)',
  4.560509554140127),
 ('Shawshank Redemption, The (1994)', 4.554557700942973),
 ('Godfather, The (1972)', 4.524966261808367),
 ('Close Shave, A (1995)', 4.52054794520548),
 ('Usual Suspects, The (1995)', 4.517106001121705),
 ("Schindler's List (1993)", 4.510416666666667),
 ('Wrong Trousers, The (1993)', 4.507936507936508)]

In [14]:
def query_edad(edad, genero_user, genero_peli):
    datos.map(lambda x: x.split("::"))\
        .filter(lambda x: userDistio)

In [ ]:
##Vamos a usar los diccionarios directamente

In [28]:
#UserID::MovieID::Rating::Timestamp
movilens.first()

'1::1193::5::978300760'

In [15]:
def parseLineN(x):
  x = x.split('::')[:3] #[1:3] me muestra los dos primeros datos del arreglo
  x[2] = float(x[2])
  return(x) # Para par clave por lo general se representa en tuplas

movilens.map(parseLineN)

PythonRDD[14] at RDD at PythonRDD.scala:53

In [25]:
Age = None
gender = None
Occupation = None
genres = None

if Age != None:
    ratings = movilens.filter(lambda x: userDictio.value[x[0]]["Age"] == Age)
    
if gender != None:
    ratings = movilens.filter(lambda x: userDictio.value[x[0]]["gender"] == gender)
    
if Occupation != None:
    ratings = movilens.filter(lambda x: userDictio.value[x[0]]["Occupation"] == Occupation)


movilens.map(lambda x: x.split("::"))\
        .map(lambda x: (userDictio.value[x[0]]["Age"],movieDictio.value[x[1]]["name"]))\
        .collect()

[('1', "One Flew Over the Cuckoo's Nest (1975)"),
 ('1', 'James and the Giant Peach (1996)'),
 ('1', 'My Fair Lady (1964)'),
 ('1', 'Erin Brockovich (2000)'),
 ('1', "Bug's Life, A (1998)"),
 ('1', 'Princess Bride, The (1987)'),
 ('1', 'Ben-Hur (1959)'),
 ('1', 'Christmas Story, A (1983)'),
 ('1', 'Snow White and the Seven Dwarfs (1937)'),
 ('1', 'Wizard of Oz, The (1939)'),
 ('1', 'Beauty and the Beast (1991)'),
 ('1', 'Gigi (1958)'),
 ('1', 'Miracle on 34th Street (1947)'),
 ('1', "Ferris Bueller's Day Off (1986)"),
 ('1', 'Sound of Music, The (1965)'),
 ('1', 'Airplane! (1980)'),
 ('1', 'Tarzan (1999)'),
 ('1', 'Bambi (1942)'),
 ('1', 'Awakenings (1990)'),
 ('1', 'Big (1988)'),
 ('1', 'Pleasantville (1998)'),
 ('1', 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 ('1', 'Back to the Future (1985)'),
 ('1', "Schindler's List (1993)"),
 ('1', 'Meet Joe Black (1998)'),
 ('1', 'Pocahontas (1995)'),
 ('1', 'E.T. the Extra-Terrestrial (1982)'),
 ('1', 'Titanic (1997)'),
 ('1', 

In [6]:
sc.stop()

# Clase 0921

In [1]:
#Se inicia el findspark
import findspark
findspark.init()

#Se configura el spark
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Pyspark")
sc = SparkContext(conf = conf)

In [2]:
def parseLines(x):
  x = x.split('::')[1:3] #[1:3] me muestra los dos primeros datos del arreglo
  x[1] = float(x[1])
  return(tuple(x)) # Para par clave por lo general se representa en tuplas

  #(reduceByKey)hace la operación en las claves (películas) que son iguales
movilens = sc.textFile("./ml-1m/ratings.dat")

ratings = movilens.map(parseLines)\
                  .mapValues(lambda x: (x,1))\
                  .reduceByKey(lambda x , y: (x[0] + y[0], x[1] + y[1]))\
                  .mapValues(lambda x: x[0]/x[1])\
                  .sortBy(lambda x: -x[1])
ratings.collect()

[('787', 5.0),
 ('1830', 5.0),
 ('3607', 5.0),
 ('3280', 5.0),
 ('989', 5.0),
 ('3881', 5.0),
 ('3233', 5.0),
 ('3172', 5.0),
 ('3656', 5.0),
 ('3382', 5.0),
 ('3245', 4.8),
 ('53', 4.75),
 ('2503', 4.666666666666667),
 ('2905', 4.608695652173913),
 ('2019', 4.560509554140127),
 ('318', 4.554557700942973),
 ('858', 4.524966261808367),
 ('745', 4.52054794520548),
 ('50', 4.517106001121705),
 ('527', 4.510416666666667),
 ('1148', 4.507936507936508),
 ('1795', 4.5),
 ('3517', 4.5),
 ('557', 4.5),
 ('2480', 4.5),
 ('2309', 4.5),
 ('578', 4.5),
 ('3888', 4.5),
 ('439', 4.5),
 ('922', 4.491489361702127),
 ('1198', 4.477724741447892),
 ('904', 4.476190476190476),
 ('1178', 4.473913043478261),
 ('260', 4.453694416583082),
 ('1212', 4.452083333333333),
 ('750', 4.4498902706656915),
 ('3338', 4.444444444444445),
 ('720', 4.426940639269406),
 ('1207', 4.425646551724138),
 ('3435', 4.415607985480944),
 ('912', 4.412822049131217),
 ('670', 4.410714285714286),
 ('2762', 4.406262708418057),
 ('3030',

In [3]:
def get_movies_info():
    #MovieID::Title::Genres
    movieNames = {}

    with open("./ml-1m/movies.dat" , "r", encoding = "latin1") as f:
        for line in f:
            x = line.split("::")
            movieNames[x[0]] = {"name" : x[1][:-7],
                                "year" : x[1][-5:-1],
                                "genres": x[2].replace("\n", "").split("|")}
    return movieNames

movieDictio = sc.broadcast(get_movies_info())

In [4]:
movieDictio.value["1"]

{'name': 'Toy Story',
 'year': '1995',
 'genres': ['Animation', "Children's", 'Comedy']}

In [5]:
def get_users_info():
    #UserID::Gender::Age::Occupation::Zip-code
    usersInfo = {}

    with open("./ml-1m/users.dat" , "r", encoding = "latin1") as f:
        for line in f:
            x = line.split("::")
            usersInfo[x[0]] = {"gender" : x[1],
                               "Age" : x[2],
                                "Occupation": x[3]}
    return usersInfo

userDictio = sc.broadcast(get_users_info())

In [6]:
userDictio.value["1"]

{'gender': 'F', 'Age': '1', 'Occupation': '10'}

In [7]:
def parseLineN(x):
  x = x.split('::')[:3] #[1:3] me muestra los dos primeros datos del arreglo
  x[2] = float(x[2])
  return(x) # Para par clave por lo general se representa en tuplas

movilens.map(parseLineN)

PythonRDD[13] at RDD at PythonRDD.scala:53

In [21]:
movilens = movilens.map(parseLineN)

In [22]:
def get_movie_recomendation(Age = None, gender = None, Occupation = None, year = None, genre = None):
    ### Filtros de usuarios
    if Age != None:
        ratings = movilens.filter(lambda x: userDictio.value[x[0]]["Age"] == Age)

    if gender != None:
        ratings = movilens.filter(lambda x: userDictio.value[x[0]]["gender"] == gender)

    if Occupation != None:
        ratings = movilens.filter(lambda x: userDictio.value[x[0]]["Occupation"] == Occupation)
    #----------------------------------------------------------

    ### Filtros de películas

    if year != None:
        ratings = movilens.filter(lambda x: movieDictio.value[x[1]]["year"] == year)

    if genre != None:
        ratings = movilens.filter(lambda x: genre in movieDictio.value[x[1]]["genres"])

    if Age == None and genre == None and Occupation == None and year == None and gender == None:
        ratings = movilens

    ### Filtrando ratings
    recomendation = ratings.map(lambda x: (x[1], x[2]))\
                           .mapValues (lambda x: (x, 1))\
                           .reduceByKey (lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                           .mapValues (lambda x: x[0]/x[1])\
                           .sortBy(lambda x: -x[1])\
                           .map(lambda x: (movieDictio.value [x[0]]["name"], x[1]))
    return (recomendation)

In [20]:
movilens.collect()

['1::1193::5::978300760',
 '1::661::3::978302109',
 '1::914::3::978301968',
 '1::3408::4::978300275',
 '1::2355::5::978824291',
 '1::1197::3::978302268',
 '1::1287::5::978302039',
 '1::2804::5::978300719',
 '1::594::4::978302268',
 '1::919::4::978301368',
 '1::595::5::978824268',
 '1::938::4::978301752',
 '1::2398::4::978302281',
 '1::2918::4::978302124',
 '1::1035::5::978301753',
 '1::2791::4::978302188',
 '1::2687::3::978824268',
 '1::2018::4::978301777',
 '1::3105::5::978301713',
 '1::2797::4::978302039',
 '1::2321::3::978302205',
 '1::720::3::978300760',
 '1::1270::5::978300055',
 '1::527::5::978824195',
 '1::2340::3::978300103',
 '1::48::5::978824351',
 '1::1097::4::978301953',
 '1::1721::4::978300055',
 '1::1545::4::978824139',
 '1::745::3::978824268',
 '1::2294::4::978824291',
 '1::3186::4::978300019',
 '1::1566::4::978824330',
 '1::588::4::978824268',
 '1::1907::4::978824330',
 '1::783::4::978824291',
 '1::1836::5::978300172',
 '1::1022::5::978300055',
 '1::2762::4::978302091',

In [23]:
recomendation = get_movie_recomendation(genre = "Comedy")
recomendation.collect()

[('Follow the Bitch', 5.0),
 ('One Little Indian', 5.0),
 ('Smashing Time', 5.0),
 ('Close Shave, A', 4.52054794520548),
 ('Wrong Trousers, The', 4.507936507936508),
 ('Yojimbo', 4.404651162790698),
 ('City Lights', 4.387453874538745),
 ('General, The', 4.368932038834951),
 ('Grand Day Out, A', 4.361522198731501),
 ('Creature Comforts', 4.335766423357664),
 ('Monty Python and the Holy Grail', 4.335209505941213),
 ('Soft Fruit', 4.333333333333333),
 ('Life Is Beautiful (La Vita è bella)', 4.329861111111111),
 ('Sting, The', 4.320305052430887),
 ('American Beauty', 4.3173862310385065),
 ('Princess Bride, The', 4.3037100949094045),
 ('Philadelphia Story, The', 4.3006872852233675),
 ('Some Like It Hot', 4.300480769230769),
 ('Palm Beach Story, The', 4.288461538461538),
 ('Cinema Paradiso', 4.287804878048781),
 ('It Happened One Night', 4.280748663101604),
 ('Cool Hand Luke', 4.253763440860215),
 ('Young Frankenstein', 4.250628667225482),
 ("Ed's Next Move", 4.25),
 ('Window to Paris', 4.25

In [24]:
!pip install flask